In [2]:
import pandas as pd
import ast
import re
from datetime import datetime

%load_ext autoreload
%autoreload 2

*   ETL - USER ITEMS 

In [3]:
user_items_ruta = 'data/australian_users_items.json'

# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Abrir el archivo y procesar cada línea
with open(user_items_ruta, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_items = pd.DataFrame(data_list)

In [4]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


A primera vista salta la columna de items, primero se va a aplicar <br>
la funcion json_normalize para "aplanar" la estructura anidada del json <br>
y tenerlo en un data frame tabular

In [6]:
# se normaliza items
df_items = pd.json_normalize(data_list, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
df_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


Se revisan los registros nulos y los duplicados

In [7]:
# Se revisan los registros nulos
df_items.isna().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
steam_id            0
items_count         0
user_id             0
user_url            0
dtype: int64

In [9]:
# Se revisan los registros duplicados
df_items.duplicated().sum()

59104

In [10]:
# Se proceden a eliminar los duplicados manteniendo el primero
df_items = df_items.drop_duplicates(keep='first')


0

Finalmente, como en la API unicamente va a ser necesario el valor de 
play time forever podemos eliminar la columna de 2weeks y aparte
nos ayuda con la optimización 

In [11]:
df_items = df_items.drop('playtime_2weeks', axis=1)

In [12]:
df_items.head()

,item_id,item_name,playtime_forever,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [13]:
items_clean_ruta = 'data/user_items_clean.csv'
df_items.to_csv(items_clean_ruta, index=False, encoding='utf-8')